In [1]:
from keras.models import load_model
import joblib
import pandas as pd
import numpy as np
import json


In [2]:
# Memuat model dan scaler yang telah disimpan
classification_model = load_model('Model/classification_model.h5')
classification_scaler = joblib.load('Model/scaler_classification.pkl')  # Memuat scaler yang sudah disimpan
regression_model  = load_model('Model/deep_learning_regression_model.h5')
regression_scaler = joblib.load('Model/scaler_deep_learning_regression.pkl')
print("Model dan scaler loaded successfully.")

Model dan scaler loaded successfully.


In [3]:

# Define a function for making predictions
def predict_rad(features, scaler, model):
    """
    Predict rad(m) value based on input weather features.

    Parameters:
        features (dict): A dictionary containing all feature values (Tn, Tx, Tavg, RH_avg, RR, ss, ff_x, ddd_x, ff_avg).
        scaler (StandardScaler): Scaler used to normalize the features.
        model (Sequential): Trained deep learning model.

    Returns:
        float: Predicted rad(m) value.
    """
     # Convert input features to a DataFrame with the same column names as the scaler expects
    feature_df = pd.DataFrame([features])
    
    # Scale the input features using the trained scaler
    scaled_features = scaler.transform(feature_df)

    # Predict rad(m) using the trained model
    predicted_value = model.predict(scaled_features)

    # Return the predicted rad(m) value
    return predicted_value[0][0]


In [4]:
def predict_condition(input_features, scaler, model):
    # Data input baru
    # new_data = np.array([[Tn, Tx, Tavg, RH_avg, ff_avg, rad_m]])
    new_data = np.array([[input_features['Tn'], input_features['Tx'], input_features['Tavg'], input_features['RH_avg'], input_features['ff_avg'], input_features['rad_m']]])

    # Normalisasi data baru menggunakan scaler yang sama seperti yang digunakan saat pelatihan
    new_data_scaled = scaler.transform(new_data)

    # Reshape data untuk input LSTM (3D tensor)
    new_data_reshaped = new_data_scaled.reshape((new_data_scaled.shape[0], 1, new_data_scaled.shape[1]))

    # Gunakan model yang dimuat untuk prediksi
    prediction = model.predict(new_data_reshaped)

    # Tampilkan hasil prediksi (nilai probabilitas, 0 atau 1)
    predicted_label = (prediction > 0.5).astype(int)
    return 'Aman' if predicted_label[0][0] == 0 else 'Tidak Aman'



In [ ]:
#tidak aman
input_regression = {
    'Tn': 28.96,      # Minimum temperature
    'Tx': 28.96,      # Maximum temperature
    'Tavg': 28.96,    # Average temperature
    'RH_avg': 89,  # Average relative humidity
    'ff_avg': 5.66,   # Average wind speed
}
#aman
# input_regression = {
#     'Tn' : 22.5,      # Minimum Temperature
#     'Tx' : 35.0,      # Maximum Temperature
#     'Tavg' : 28.0 ,   # Average Temperature
#     'RH_avg' : 75.0,  # Average Relative Humidityp
#     'ff_avg' : 3.0 ,  # Average Wind Speed
# }

# Call the prediction function
predicted_rad = predict_rad(input_regression, regression_scaler, regression_model)
print(f"Predicted rad(m): {predicted_rad:.4f}")

#tidak aman
input_classification = {
    'Tn': 28.96,      # Minimum temperature
    'Tx': 28.96,      # Maximum temperature
    'Tavg': 28.96,    # Average temperature
    'RH_avg': 89,  # Average relative humidity
    'ff_avg': 5.66,   # Average wind speed
    'rad_m': predicted_rad
}
#aman
# input_classification = {
#    'Tn' : 22.5,      # Minimum Temperature
#     'Tx' : 35.0,      # Maximum Temperature
#     'Tavg' : 28.0 ,   # Average Temperature
#     'RH_avg' : 75.0,  # Average Relative Humidityp
#     'ff_avg' : 3.0 ,  # Average Wind Speed
#     'rad_m': predicted_rad
# }
# Call the prediction function
predicted_condition = predict_condition(input_classification, classification_scaler, classification_model)
print(f"Predicted condition: {predicted_condition}")

# Create a dictionary with the results
result = {
    'predicted_rad': float(predicted_rad),
    'condition': predicted_condition
}

# Write the result to a JSON file
with open('prediction_result.json', 'w') as json_file:
    json.dump(result, json_file)

print("Results saved to prediction_result.json")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
Predicted rad(m): -0.2025


c:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step
Predicted condition: Tidak Aman
Results saved to prediction_result.json
